In [119]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [120]:
files = glob.glob("train/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)

C:\Users\hiro2\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 651975 entries, 1117225 to 47001939
Data columns (total 27 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   種類            651975 non-null  object 
 1   地域            0 non-null       float64
 2   市区町村コード       651975 non-null  int64  
 3   都道府県名         651975 non-null  object 
 4   市区町村名         651975 non-null  object 
 5   地区名           651318 non-null  object 
 6   最寄駅：名称        649342 non-null  object 
 7   最寄駅：距離（分）     628916 non-null  object 
 8   間取り           629546 non-null  object 
 9   面積（㎡）         651975 non-null  object 
 10  土地の形状         0 non-null       float64
 11  間口            0 non-null       float64
 12  延床面積（㎡）       0 non-null       float64
 13  建築年           633280 non-null  object 
 14  建物の構造         637098 non-null  object 
 15  用途            601333 non-null  object 
 16  今後の利用目的       288948 non-null  object 
 17  前面道路：方位       0 non-null       float64
 

In [122]:
df.head()

,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,,,,
1117225,中古マンション等,NaN,1213,北海道,苫小牧市,新中野町,苫小牧,25,３ＬＤＫ,120,...,NaN,NaN,NaN,準工業地域,60.0,200.0,2018年第３四半期,未改装,NaN,7.431364
1001180,中古マンション等,NaN,1101,北海道,札幌市中央区,北２条西,西２８丁目,8,４ＬＤＫ,150,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2018年第３四半期,未改装,NaN,7.568202
1099061,中古マンション等,NaN,1207,北海道,帯広市,西３条南,帯広,2,２ＬＤＫ,80,...,NaN,NaN,NaN,商業地域,80.0,500.0,2016年第２四半期,未改装,NaN,7.204120
1039968,中古マンション等,NaN,1105,北海道,札幌市豊平区,平岸２条,南平岸,6,３ＬＤＫ,90,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2018年第４四半期,改装済,NaN,7.447158
1072923,中古マンション等,NaN,1202,北海道,函館市,梁川町,五稜郭,30分?60分,２ＬＤＫ,70,...,NaN,NaN,NaN,商業地域,80.0,400.0,2015年第１四半期,未改装,NaN,7.322219


In [123]:
def data_pre(df):

    nonnull_list = []

    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
            nonnull_list.append(col)

    df = df.drop(nonnull_list, axis=1)

    df = df.drop("市区町村名", axis=1)

    df = df.drop("種類", axis=1)

    dis = {
        "30分?60分" : 45,
        "1H?1H30" : 75,
        "2H?" : 120,
        "1H30?2H": 105,
    }

    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)

    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[1].split("年")[0])
            year = 33 - num
        if "令和" in i:
            num = float(i.split("令和")[1].split("年")[0])
            year = 3 - num
        if "昭和" in i:
            num = float(i.split("昭和")[1].split("年")[0])
            year = 96 - num
        y_list[i] = year
    y_list["戦前"] = 76
    df["建築年"] = df["建築年"].replace(y_list)

    year = {
        "年第１四半期": ".25",
        "年第２四半期": ".50",
        "年第３四半期": ".75",
        "年第４四半期": ".99"
    }
    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)
        year_list[i] = year_rep
    year_list
    df["取引時点"] = df["取引時点"].replace(year_list).astype(float)
    
    for col in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
        df[col] = df[col].astype("category")
    
    return df

In [124]:
df = data_pre(df)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 651975 entries, 1117225 to 47001939
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   市区町村コード       651975 non-null  int64   
 1   都道府県名         651975 non-null  category
 2   地区名           651318 non-null  category
 3   最寄駅：名称        649342 non-null  category
 4   最寄駅：距離（分）     628916 non-null  float64 
 5   間取り           629546 non-null  category
 6   面積（㎡）         651975 non-null  float64 
 7   建築年           633280 non-null  float64 
 8   建物の構造         637098 non-null  category
 9   用途            601333 non-null  category
 10  今後の利用目的       288948 non-null  category
 11  都市計画          633162 non-null  category
 12  建ぺい率（％）       629168 non-null  float64 
 13  容積率（％）        629168 non-null  float64 
 14  取引時点          651975 non-null  float64 
 15  改装            593301 non-null  category
 16  取引の事情等        18397 non-null   category
 17  取引価格（総額）_log  651975 

,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,
1117225,1213,北海道,新中野町,苫小牧,25.0,３ＬＤＫ,120.0,12.0,ＲＣ,住宅,住宅,準工業地域,60.0,200.0,2018.75,未改装,NaN,7.431364
1001180,1101,北海道,北２条西,西２８丁目,8.0,４ＬＤＫ,150.0,19.0,ＳＲＣ,NaN,住宅,近隣商業地域,80.0,300.0,2018.75,未改装,NaN,7.568202
1099061,1207,北海道,西３条南,帯広,2.0,２ＬＤＫ,80.0,19.0,ＲＣ,住宅,住宅,商業地域,80.0,500.0,2016.50,未改装,NaN,7.204120
1039968,1105,北海道,平岸２条,南平岸,6.0,３ＬＤＫ,90.0,24.0,NaN,住宅,住宅,近隣商業地域,80.0,300.0,2018.99,改装済,NaN,7.447158
1072923,1202,北海道,梁川町,五稜郭,45.0,２ＬＤＫ,70.0,9.0,ＲＣ,住宅,住宅,商業地域,80.0,400.0,2015.25,未改装,NaN,7.322219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47006116,47211,沖縄県,泡瀬,NaN,NaN,３ＬＤＫ,115.0,15.0,ＲＣ,住宅,住宅,第１種中高層住居専用地域,60.0,200.0,2016.50,未改装,NaN,7.230449
47016927,47201,沖縄県,宇栄原,赤嶺,7.0,３ＬＤＫ,65.0,30.0,ＲＣ,住宅,NaN,NaN,NaN,NaN,2011.75,改装済,NaN,7.176091
47027045,47201,沖縄県,おもろまち,おもろまち,12.0,２ＬＤＫ,65.0,22.0,ＲＣ,住宅,NaN,商業地域,80.0,400.0,2009.25,改装済,NaN,7.477121


In [125]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

# df_train, df_val = train_test_split(df, test_size=0.2)

df_train = df[df["取引時点"] < 2019.99]
df_val = df[df["取引時点"] >= 2019.99]

col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
    "objective": "regression",
    "metrics": "mae"
}

model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100, verbose_eval=-1)


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.040584 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11643
[LightGBM] [Info] Number of data points in the train set: 625333, number of used features: 17
[LightGBM] [Info] Start training from score 7.216098
Training until validation scores don't improve for 100 rounds
C:\Users\hiro2\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\hiro2\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
Did not meet early stopping. Best iteration is:
[992]	valid_0's l1: 0.0791934


In [126]:
vals = model.predict(val_x)
mae(vals, val_y)

0.07919337738280226

In [127]:
df_test = pd.read_csv("test.csv", index_col=0)

df_test = data_pre(df_test)

In [128]:
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("submit_submission.csv")

In [129]:
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
地区名,9199
最寄駅：名称,6519
建築年,3071
取引時点,2899
面積（㎡）,2107
最寄駅：距離（分）,1863
間取り,866
市区町村コード,747
都道府県名,570
改装,525
